In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time


In [7]:
# 📌 Veri setini oku
file_path = "CXFeatured.xlsx"
df = pd.read_excel(file_path)

# 📌 Hedef değişkeni belirle
target_column = "Attrition_Flag_Existing Customer"
X = df.drop(columns=[target_column])
y = df[target_column]

# 📌 Eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 📌 One-Hot Encoding uygulanacak sütunları belirle
ohe_cols = [col for col in X_train.columns if X_train[col].nunique() == 2]
exclude_cols = ["Avg_Utilization_Ratio"]

# 📌 StandardScaler uygulanacak sütunları belirle
scaled_features = [col for col in X_train.columns if col not in ohe_cols and col not in exclude_cols]

# 📌 StandardScaler uygula (sadece sürekli sayısal değişkenler)
scaler = StandardScaler()
X_trainS = X_train.copy()
X_testS = X_test.copy()
X_trainS[scaled_features] = scaler.fit_transform(X_train[scaled_features])
X_testS[scaled_features] = scaler.transform(X_test[scaled_features])

# 📌 Numpy array formatına çevir
X_trainS = X_trainS.to_numpy()
X_testS = X_testS.to_numpy()


In [9]:
# 📌 Hiperparametreleri tanımla
dense_layers = [1, 2, 3]  # Gizli katman sayısı
neuron_sizes = [32, 64, 128]  # Katman başına nöron sayısı
dropout_rates = [0.1, 0.2, 0.3]  # Dropout oranı
learning_rates = [0.01, 0.05, 0.1]  # Öğrenme oranı

# 📌 Modeli eğitme süreci
input_shape = (X_trainS.shape[1],)  # Giriş boyutu

for dense_layer in dense_layers:
    for neuron_size in neuron_sizes:
        for dropout_rate in dropout_rates:
            for learning_rate in learning_rates:
                model = Sequential([
                    BatchNormalization(input_shape=input_shape),
                    Dense(neuron_size, activation='relu')
                ])

                for _ in range(dense_layer):
                    model.add(BatchNormalization())
                    model.add(Dense(neuron_size, activation='relu'))
                    model.add(BatchNormalization())
                    model.add(Dropout(dropout_rate))  # Overfitting önlemek için dropout ekle

                model.add(Dense(1, activation='sigmoid'))  # Çıkış katmanı (Binary Classification)

                #  TensorBoard için isimlendirme
                name = "DL{}_LS{}_DRO{}_LR{}".format(dense_layer, neuron_size, dropout_rate, learning_rate)
                tensorboard = TensorBoard(log_dir="logs/new{}".format(name))

                #  Modeli derle
                model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['binary_accuracy'])

                #  Modeli eğit
                model.fit(X_trainS, y_train, validation_data=(X_testS, y_test), epochs=20, callbacks=[tensorboard])


Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - binary_accuracy: 0.5935 - loss: 0.7096 - val_binary_accuracy: 0.8131 - val_loss: 0.5063
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - binary_accuracy: 0.7951 - loss: 0.4474 - val_binary_accuracy: 0.8318 - val_loss: 0.4651
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.8501 - loss: 0.3585 - val_binary_accuracy: 0.8411 - val_loss: 0.4429
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - binary_accuracy: 0.8541 - loss: 0.3239 - val_binary_accuracy: 0.8598 - val_loss: 0.3971
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.8809 - loss: 0.2819 - val_binary_accuracy: 0.8879 - val_loss: 0.3597
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - binary_accuracy: 0.9055 - loss: 0.2629 - val_binary_accuracy: 0.8598 - val_loss: 0.3221
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - binary_accuracy: 0.9178 - loss: 0.2285 - val_binary_accuracy: 0.8598 - val_loss: 0.3232
Epoch 8/20
14/14 ━━

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score

# 📌 Modeli oluştur
model = Sequential([
    BatchNormalization(input_shape=(X_trainS.shape[1],)),  # Giriş verisini normalize et
    Dense(64, activation='relu'),  # İlk gizli katman
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),  # İkinci gizli katman
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Çıkış katmanı (Binary Classification)
])

# 📌 Modeli derle
optimizer = Adam(learning_rate=0.05)  # Öğrenme oranı
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])

# 📌 Modeli eğit
history = model.fit(X_trainS, y_train, validation_data=(X_testS, y_test), epochs=50, batch_size=32, verbose=1)

# 📌 Modelin test seti üzerindeki tahminlerini al
y_pred_probs = model.predict(X_testS)
y_pred = (y_pred_probs > 0.5).astype(int)

# 📌 Precision, Recall, F1 Score hesapla
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 📌 Sonuçları ekrana yazdır
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")



c:\Users\EMRE\anaconda3\Lib\site-packages\keras\src\layers\normalization\batch_normalization.py:142: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - binary_accuracy: 0.6299 - loss: 0.9003 - val_binary_accuracy: 0.7757 - val_loss: 0.5645
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.7865 - loss: 0.4679 - val_binary_accuracy: 0.8692 - val_loss: 0.3217
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.8388 - loss: 0.3430 - val_binary_accuracy: 0.8598 - val_loss: 0.4043
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.8081 - loss: 0.3642 - val_binary_accuracy: 0.8879 - val_loss: 0.2919
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.8850 - loss: 0.2900 - val_binary_accuracy: 0.8505 - val_loss: 0.2923
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.8720 - loss: 0.2747 - val_binary_accuracy: 0.8598 - val_loss: 0.4357
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.8531 - loss: 0.3173 - val_binary_accuracy: 0.8785 - val_loss: 0.3363
Epoch 8/50
14/14 ━━